In [2]:
import flor
from app import config as c

In [3]:
infer = flor.dataframe(c.first_page, c.page_path)
infer

,document,page,projid,tstamp,filename,first_page,page_path
0,1,1,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,1,app/static/private/imgs/6-Complaint__Judgment/...
1,1,2,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/6-Complaint__Judgment/...
2,1,3,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/6-Complaint__Judgment/...
3,1,4,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/6-Complaint__Judgment/...
4,1,5,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/6-Complaint__Judgment/...
...,...,...,...,...,...,...,...
229,6,52,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/george_santos_report/p...
230,6,53,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/george_santos_report/p...
231,6,54,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/george_santos_report/p...
232,6,55,pdf_parser,2024-02-13 13:24:18,label_by_hand.py,0,app/static/private/imgs/george_santos_report/p...


In [ ]:
webapp = flor.utils.latest(flor.dataframe(c.pdf_name, c.page_color))
webapp

In [ ]:
import os

# get basename of page_path
infer['page_path'] = infer['page_path'].map(lambda x: os.path.basename(os.path.dirname(x)))
infer

In [ ]:
webapp['pdf_name'] = webapp['pdf_name'].map(lambda x: os.path.splitext(x)[0])
webapp

In [ ]:
merged_df = webapp.merge(infer, left_on=['projid', 'pdf_name', 'page'], right_on=['projid','page_path', 'page'])
merged_df.columns

In [ ]:
selected_cols = ['projid',  'pdf_name', 'page',  'tstamp_x', 'filename_x', 'tstamp_y', 'filename_y',  'first_page', 'page_color']
merged_df = merged_df[selected_cols]
merged_df

In [ ]:
merged_df['page_color'].astype(int).to_numpy()

In [ ]:
(merged_df['page_color'].shift() != merged_df['page_color']).astype(int).to_numpy()

In [19]:
merged_df['first_page_infer'] = merged_df['first_page']
merged_df['first_page_webapp'] = (merged_df['page_color'].shift() != merged_df['page_color']).astype(int).to_numpy()

In [ ]:
merged_df

In [ ]:
any(merged_df['tstamp_x'].drop_duplicates() > merged_df['tstamp_y'].drop_duplicates())

In [ ]:
# Check timestamps,
if any(merged_df['tstamp_x'].drop_duplicates() > merged_df['tstamp_y'].drop_duplicates()):
    # if the timestamp of webapp is greater, return first_page_webapp
    merged_df['first_page'] = merged_df['first_page_webapp']
else:
    # else return first_page_infer
    merged_df['first_page'] = merged_df['first_page_infer']

merged_df